In [3]:
import os
import pandas as pd

In [4]:
csv_path = '../content/sample_data'
cleandf = pd.read_csv(os.path.join(csv_path, 'clean_data.csv'))

In [5]:
cleandf

,Unnamed: 0,text,target
0,0,smell hillary fear daniel greenfield shillman ...,0
1,1,watch exact moment paul ryan commit political ...,0
2,2,kerry go paris gesture sympathy u secretary st...,1
3,3,bernie supporter twitter erupt anger dnc try w...,0
4,4,battle new york primary matter primary day new...,1
...,...,...,...
6330,6330,state department say can find email clinton sp...,1
6331,6331,p pb stand plutocratic pentagon p pb stand plu...,0
6332,6332,anti trump protester tool oligarchy informatio...,0
6333,6333,ethiopia obama seek progress peace security ea...,1


In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Conv1D, Embedding, Flatten, Masking, Dropout, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [57]:
tokenizer = Tokenizer()
model = Sequential()
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
X_train, X_test, y_train, y_test = train_test_split(cleandf['text'], cleandf['target'], test_size=0.25)

In [58]:
tokenizer.fit_on_texts(X_train)

In [59]:
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [60]:
X_train_token = pad_sequences(X_train_token, dtype='int32', padding='post', maxlen=300)
X_test_token = pad_sequences(X_test_token, dtype='int32', padding='post', maxlen=300)

In [61]:
model.add(Embedding(
    input_dim=(len(tokenizer.word_index))+1, # +1 for the 0 padding
    input_length=300, # Max_sentence_length (optional, for model summary)
    output_dim=100,
    mask_zero=True, # Built-in masking layer :)
))
model.add(LSTM(50, return_sequences=False))
#model.add(Conv1D(30, kernel_size=5, activation='tanh'))
#model.add(Flatten())
model.add(Dense(25, activation='relu'))
#model.add(Dropout(rate=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=rms,
             metrics=['accuracy'])

In [63]:
model.fit(X_train_token, y_train, 
          epochs=50, 
          batch_size=32,
          validation_split=0.25,
          callbacks=[es],
        use_multiprocessing=True
         )

Epoch 1/50
112/112 [==============================] - 39s 304ms/step - loss: 0.6901 - accuracy: 0.5824 - val_loss: 0.6855 - val_accuracy: 0.6734
Epoch 2/50
112/112 [==============================] - 32s 285ms/step - loss: 0.6721 - accuracy: 0.7143 - val_loss: 0.6471 - val_accuracy: 0.7214
Epoch 3/50
112/112 [==============================] - 32s 284ms/step - loss: 0.5238 - accuracy: 0.8288 - val_loss: 0.4490 - val_accuracy: 0.8712
Epoch 4/50
112/112 [==============================] - 34s 305ms/step - loss: 0.4277 - accuracy: 0.8745 - val_loss: 0.4042 - val_accuracy: 0.8611
Epoch 5/50
112/112 [==============================] - 32s 284ms/step - loss: 0.3410 - accuracy: 0.8978 - val_loss: 0.3476 - val_accuracy: 0.8746
Epoch 6/50
112/112 [==============================] - 32s 284ms/step - loss: 0.2450 - accuracy: 0.9335 - val_loss: 0.2854 - val_accuracy: 0.8981
Epoch 7/50
112/112 [==============================] - 32s 283ms/step - loss: 0.1863 - accuracy: 0.9554 - val_loss: 0.2505 - val_ac

In [64]:
model.evaluate(X_test_token, y_test, verbose=1)

50/50 [==============================] - 3s 58ms/step - loss: 0.2631 - accuracy: 0.9173


[0.2631131112575531, 0.9172979593276978]

In [66]:
model = Sequential()
model.add(Embedding(
    input_dim=(len(tokenizer.word_index))+1, # +1 for the 0 padding
    input_length=300, # Max_sentence_length (optional, for model summary)
    output_dim=100,
    mask_zero=True, # Built-in masking layer :)
))
model.add(GRU(50, return_sequences=False))
#model.add(Conv1D(30, kernel_size=5, activation='tanh'))
#model.add(Flatten())
model.add(Dense(25, activation='relu'))
#model.add(Dropout(rate=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=rms,
             metrics=['accuracy'])

In [67]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 300, 100)          4761600   
                                                                 
 gru (GRU)                   (None, 50)                22800     
                                                                 
 dense_16 (Dense)            (None, 25)                1275      
                                                                 
 dense_17 (Dense)            (None, 1)                 26        
                                                                 
Total params: 4,785,701
Trainable params: 4,785,701
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.fit(X_train_token, y_train, 
          epochs=50, 
          batch_size=32,
          validation_split=0.25,
          callbacks=[es],
        use_multiprocessing=True
         )

Epoch 1/50
112/112 [==============================] - 35s 273ms/step - loss: 0.6899 - accuracy: 0.6043 - val_loss: 0.6875 - val_accuracy: 0.6322
Epoch 2/50
112/112 [==============================] - 30s 268ms/step - loss: 0.6798 - accuracy: 0.7118 - val_loss: 0.6753 - val_accuracy: 0.6835
Epoch 3/50
112/112 [==============================] - 31s 273ms/step - loss: 0.6560 - accuracy: 0.7609 - val_loss: 0.6456 - val_accuracy: 0.7104
Epoch 4/50
112/112 [==============================] - 30s 272ms/step - loss: 0.6006 - accuracy: 0.7993 - val_loss: 0.5766 - val_accuracy: 0.7559
Epoch 5/50
112/112 [==============================] - 29s 263ms/step - loss: 0.4544 - accuracy: 0.8324 - val_loss: 0.3924 - val_accuracy: 0.8308
Epoch 6/50
112/112 [==============================] - 31s 275ms/step - loss: 0.3072 - accuracy: 0.8835 - val_loss: 0.3351 - val_accuracy: 0.8510
Epoch 7/50
112/112 [==============================] - 30s 265ms/step - loss: 0.2424 - accuracy: 0.9065 - val_loss: 0.2884 - val_ac

In [69]:
model.evaluate(X_test_token, y_test, verbose=1)

50/50 [==============================] - 3s 51ms/step - loss: 0.2519 - accuracy: 0.9122


[0.25192204117774963, 0.9122474789619446]

# Attention: Beggining of Word2Vec
## Better run only on Google Colab

In [15]:
from gensim.models import Word2Vec
import numpy as np
#import gensim.downloader as api

In [35]:
#word2vec = api.load('glove-wiki-gigaword-50') # this one crashes my kernel everytime
#word2vec = api.load('text8') # this one takes an eternity
word2vec = Word2Vec(sentences=X_train)

In [17]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

In [18]:
# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [36]:
# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)

In [37]:
X_train_embed = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=300)
X_test_embed = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=300)

In [38]:
for X in [X_train_embed, X_test_embed]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.vector_size


assert X_train_embed.shape[0] == len(X_train)
assert X_test_embed.shape[0] == len(X_test)

In [28]:
rms = RMSprop(learning_rate=0.0001)

In [39]:
model = Sequential()
model.add(Masking(mask_value=0, input_shape=X_train_embed.shape[1:]))
#model.add(LSTM(20, return_sequences=True))
model.add(Conv1D(30, kernel_size=5, activation='tanh'))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
#model.add(Dropout(rate=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=rms,
             metrics=['accuracy'])

In [40]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_3 (Masking)         (None, 300, 100)          0         
                                                                 
 conv1d_4 (Conv1D)           (None, 296, 30)           15030     
                                                                 
 flatten_4 (Flatten)         (None, 8880)              0         
                                                                 
 dense_8 (Dense)             (None, 15)                133215    
                                                                 
 dense_9 (Dense)             (None, 1)                 16        
                                                                 
Total params: 148,261
Trainable params: 148,261
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.fit(X_train_embed, y_train, 
          epochs=50, 
          batch_size=32,
          validation_split=0.25,
          callbacks=[es],
        use_multiprocessing=True
         )

Epoch 1/50
112/112 [==============================] - 6s 46ms/step - loss: 0.6907 - accuracy: 0.5260 - val_loss: 0.6740 - val_accuracy: 0.5715
Epoch 2/50
112/112 [==============================] - 5s 45ms/step - loss: 0.6333 - accuracy: 0.6663 - val_loss: 0.6643 - val_accuracy: 0.5606
Epoch 3/50
112/112 [==============================] - 5s 45ms/step - loss: 0.5770 - accuracy: 0.7373 - val_loss: 0.6510 - val_accuracy: 0.6094
Epoch 4/50
112/112 [==============================] - 5s 46ms/step - loss: 0.5244 - accuracy: 0.7915 - val_loss: 0.6508 - val_accuracy: 0.6044
Epoch 5/50
112/112 [==============================] - 5s 45ms/step - loss: 0.4756 - accuracy: 0.8266 - val_loss: 0.6514 - val_accuracy: 0.6136
Epoch 6/50
112/112 [==============================] - 5s 44ms/step - loss: 0.4308 - accuracy: 0.8585 - val_loss: 0.6578 - val_accuracy: 0.6077
Epoch 7/50
112/112 [==============================] - 5s 45ms/step - loss: 0.3900 - accuracy: 0.8872 - val_loss: 0.6651 - val_accuracy: 0.6094

In [42]:
model.evaluate(X_test_embed, y_test, verbose=1)

50/50 [==============================] - 1s 15ms/step - loss: 1.6161 - accuracy: 0.6162


[1.616072416305542, 0.6161616444587708]